In [1]:
# Directories
import os

new_directory = r'c://Users//Fer//TESIS_ARCHIVOS//TESIS_AIRE//MP_Forecasting//aqi_forecasting//notebooks'
os.chdir(new_directory)

# Data Manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation


# Training utils
from training_code.utils import utils_svr

# Optuna
import optuna
import joblib
import pickle

# Tiempo
import datetime as dt
from dateutil.relativedelta import relativedelta, MO

# Modelos
from sklearn.linear_model import LinearRegression # for building a linear regression model
from sklearn.svm import SVR # for building SVR model
from sklearn.preprocessing import MinMaxScaler
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Metricas
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_absolute_percentage_error #MAPE
from sklearn.metrics import mean_squared_error #MSE, para RMSE: squared = False

# Visualizations
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization
import matplotlib.pyplot as plt

# Advertencias
import warnings
warnings.filterwarnings("ignore")

c:\Users\Fer\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
datos = pd.read_csv('datos/230127_train_ESTACIONES.csv', parse_dates=['FECHAHORA'])

# VENTANAS DE TIEMPO

In [ ]:
horizon = [0.25, 0.5, 1]
resample = [5, 30, 60]
previous = [2, 7, 15]

# Start resampling

for resample_item in resample:

    df_train = datos.copy()

    df_train.drop(['ANHO', 'DIA', 'MES', 'HORA', 'MINUTO', 'DIA_TRAF_COD', 'TRAFICO_COD', 'TIPO_COD',
                'MEDICION_DIA', 'MP1_ANTERIOR', 'MP2_5_ANTERIOR', 'MP10_ANTERIOR',
                'TEMPERATURA_PRONOSTICO', 'HUMEDAD_PRONOSTICO', 'PRESION_PRONOSTICO'], axis = 1, inplace = True)

    df_train = df_train.set_index('FECHAHORA', drop = True)

    lista_resample = []

    if resample_item != 5: 

        r_i = str(resample_item) + 'T'

        for station in range(1,11):
            df_aux = df_train[df_train['ESTACION'] == station]

            df_aux = df_aux.resample(r_i).mean()
            lista_resample.append(df_aux)

        df_train = pd.concat(lista_resample)

    df_train['ANHO'] = df_train.index.year
    df_train['MES'] = df_train.index.month
    df_train['HORA'] = df_train.index.hour
    df_train['MINUTO'] = df_train.index.minute
    df_train['DIA'] = df_train.index.day
    df_train['FECHAHORA'] = df_train.index
    df_train.reset_index(inplace = True, drop = True)

    df_train.to_csv('datos/experimento_full/train_resample_'+str(resample_item)+'_minutes.csv')

    # OPTUNA

    for previous_item in previous:
        for horizon_item in horizon:
            
            estacion = 4

            variables = ["ANHO", 'DIA', 'MES', 'HORA', 'MINUTO', 'MP1', 'MP2_5', 'MP10', 'AQI_MP10', 'AQI_MP2_5', 'TIPO', 'TRAFICO', 'HUMEDAD', 
                        'PRESION', 'TEMPERATURA','DIA_SEM']

            dependent = ['AQI_MP2_5']

            number_of_features = len(variables)

            training_days = previous_item
            forecast_days = horizon_item

            if resample_item == 5:
                samples_per_day = 288
            elif resample_item == 30:
                samples_per_day = 48
            elif resample_item == 60:
                samples_per_day = 24

            step = forecast_days*samples_per_day

            train_months = relativedelta(months = 12)

            input_samples = int(samples_per_day * training_days) # cantidad de muestras en 7 dias
            output_samples = int(samples_per_day * forecast_days) # cantidad de muestras en 1 dia
            train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test

            X_train, y_train, X_test, y_test = utils_svr.get_everything(df_train, 
                                                                estacion,
                                                                train_months, 
                                                                variables, 
                                                                dependent, 
                                                                train_test_samples, 
                                                                input_samples, 
                                                                output_samples, 
                                                                number_of_features,
                                                                step)
            
            def objective(trial):
    
                params = {
                    'max_depth': trial.suggest_int('max_depth', 1, 9),
                    'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
                    'n_estimators': trial.suggest_int('n_estimators', 10, 300),
                    'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                    'gamma': trial.suggest_loguniform('gamma', 1e-5, 1.0),
                    'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
                    'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
                }
                
                optuna_xgb_model = xgb.XGBRegressor(** params)
                
                trained_xgb_model = MultiOutputRegressor(optuna_xgb_model).fit(X_train , y_train)

                prediction = trained_xgb_model.predict(X_test)

                MAPE = mean_absolute_percentage_error(prediction, y_test)
                print('MAPE: ', MAPE)
                MAE = np.mean(np.abs(prediction - y_test))
                print('MAE: ', MAE)

                MSE = mean_squared_error(prediction, y_test)
                
                return MSE

            study = optuna.create_study(direction='minimize')

            study.optimize(objective, n_trials = 100 )

            trial = study.best_trial

            joblib.dump(study, "optuna_studies/XGBOOST/experimento_full/study_XGBOOST_mejoresparam20_resample_"+str(resample_item)+"_horizon_"+str(horizon_item)+"_previous_"+str(previous_item)+".pkl")


            # TESTEO

            # trained_svr_model = joblib.load('models/models_xgboost/experimento_full/xgboost_estacion_4_resample_'+str(resample_item)+'_horizon_'+str(horizon_item)+'_previous_'+str(previous_item)+'.pkl')

            prediction = trained_xgb_model.predict(X_test)
            
            predicciones = {}
            metricas = {}
            # guardamos los valores predecidos vs reales en un diccionario
            
            predicciones[4] = {'real' : y_test, 'prediccion': prediction}


            mean_real = y_test.mean()
            mean_prediction = prediction.mean()

            MAPE = mean_absolute_percentage_error(prediction, y_test) * 100
            MAE = mean_absolute_error(prediction, y_test)
            RMSE = mean_squared_error(prediction, y_test, squared = False)
            
            # guardamos las metricas en un diccionario
            
            metricas[4] = {'MAE': MAE, "MAPE": MAPE, 'RMSE': RMSE, 'Media real' : mean_real, 'Media predecida': mean_prediction}

            df_metricas = pd.DataFrame.from_dict(metricas)

            df_metricas.to_csv('metrics/SVR/experimento_full/testing_svr_experimento_full_resample_'+str(resample_item)+'_horizon_'+str(horizon_item)+'_previous_'+str(previous_item)+'.csv')

            list_dfs = []


            d = {'TARGET': predicciones[4]['real'].flatten(), 'FORECAST': predicciones[4]['prediccion'].flatten()}
            df_predicciones = pd.DataFrame(data = d)
            df_predicciones['ESTACION'] = 4

            df_predicciones.to_csv('datos/experimento_full/predicciones_SVR_estacion4_resample_'+str(resample_item)+'_horizon_'+str(horizon_item)+'_previous_'+str(previous_item)+'.csv')

            print('testing_'+ str(resample_item)+'_'+str(horizon_item)+'_'+ str(previous_item)+' done!')

# MEJORES PARAMETROS / 10% DE CORRELACION

In [9]:
horizon = [0.25, 0.5, 1]
resample = [5, 30, 60]
previous = [2, 7, 15]

# Start resampling

for resample_item in resample:

    df_train = datos.copy()

    df_train.drop(['ANHO', 'DIA', 'MES', 'HORA', 'MINUTO', 'DIA_TRAF_COD', 'TRAFICO_COD', 'TIPO_COD',
                'MEDICION_DIA', 'MP1_ANTERIOR', 'MP2_5_ANTERIOR', 'MP10_ANTERIOR',
                'TEMPERATURA_PRONOSTICO', 'HUMEDAD_PRONOSTICO', 'PRESION_PRONOSTICO'], axis = 1, inplace = True)

    df_train = df_train.set_index('FECHAHORA', drop = True)

    lista_resample = []

    if resample_item != 5: 

        r_i = str(resample_item) + 'T'

        for station in range(1,11):
            df_aux = df_train[df_train['ESTACION'] == station]

            df_aux = df_aux.resample(r_i).mean()
            lista_resample.append(df_aux)

        df_train = pd.concat(lista_resample)

    df_train['ANHO'] = df_train.index.year
    df_train['MES'] = df_train.index.month
    df_train['HORA'] = df_train.index.hour
    df_train['MINUTO'] = df_train.index.minute
    df_train['DIA'] = df_train.index.day
    df_train['FECHAHORA'] = df_train.index
    df_train.reset_index(inplace = True, drop = True)

    df_train.to_csv('datos/experimento_full/train_resample_'+str(resample_item)+'_minutes.csv')

    # OPTUNA

    for previous_item in previous:
        for horizon_item in horizon:
            
            estacion = 4

            variables = ['MES', 'MP1', 'MP2_5', 'MP10', 'AQI_MP10', 'AQI_MP2_5', 'HUMEDAD', 
                        'PRESION', 'TEMPERATURA','DIA_SEM']

            dependent = ['AQI_MP2_5']

            number_of_features = len(variables)

            training_days = previous_item
            forecast_days = horizon_item

            if resample_item == 5:
                samples_per_day = 288
            elif resample_item == 30:
                samples_per_day = 48
            elif resample_item == 60:
                samples_per_day = 24

            step = forecast_days*samples_per_day

            train_months = relativedelta(months = 12)

            input_samples = int(samples_per_day * training_days) # cantidad de muestras en 7 dias
            output_samples = int(samples_per_day * forecast_days) # cantidad de muestras en 1 dia
            train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test

            X_train, y_train, X_test, y_test = utils_xgboost.get_everything(df_train, 
                                                                estacion,
                                                                train_months, 
                                                                variables, 
                                                                dependent, 
                                                                train_test_samples, 
                                                                input_samples, 
                                                                output_samples, 
                                                                number_of_features,
                                                                step)
            
            def objective(trial):
    
                params = {
                    'max_depth': trial.suggest_int('max_depth', 1, 9),
                    'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
                    'n_estimators': trial.suggest_int('n_estimators', 10, 300),
                    'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                    'gamma': trial.suggest_loguniform('gamma', 1e-5, 1.0),
                    'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
                    'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
                }
                
                optuna_xgb_model = xgb.XGBRegressor(** params)
                
                trained_xgb_model = MultiOutputRegressor(optuna_xgb_model).fit(X_train , y_train)

                prediction = trained_xgb_model.predict(X_test)

                MAPE = mean_absolute_percentage_error(prediction, y_test)
                print('MAPE: ', MAPE)
                MAE = np.mean(np.abs(prediction - y_test))
                print('MAE: ', MAE)

                MSE = mean_squared_error(prediction, y_test)
                
                return MSE

            study = optuna.create_study(direction='minimize')

            study.optimize(objective, n_trials = 100 )

            trial = study.best_trial

            joblib.dump(study, "optuna_studies/XGBOOST/experimento_full/study_XGBOOST_mejoresparam20_resample_"+str(resample_item)+"_horizon_"+str(horizon_item)+"_previous_"+str(previous_item)+".pkl")


            # TESTEO

            # trained_xgb_model = joblib.load('models/models_xgboost/experimento_full/xgboost_mejoresparam_estacion_4_resample_'+str(resample_item)+'_horizon_'+str(horizon_item)+'_previous_'+str(previous_item)+'.pkl')

            prediction = trained_xgb_model.predict(X_test)
            
            predicciones = {}
            metricas = {}
            # guardamos los valores predecidos vs reales en un diccionario
            
            predicciones[4] = {'real' : y_test, 'prediccion': prediction}


            mean_real = y_test.mean()
            mean_prediction = prediction.mean()

            MAPE = mean_absolute_percentage_error(prediction, y_test)
            MAE = mean_absolute_error(prediction, y_test)
            RMSE = mean_squared_error(prediction, y_test, squared = False)
            
            # guardamos las metricas en un diccionario
            
            metricas[4] = {'MAE': MAE, "MAPE": MAPE, 'RMSE': RMSE, 'Media real' : mean_real, 'Media predecida': mean_prediction}

            df_metricas = pd.DataFrame.from_dict(metricas)

            df_metricas.to_csv('metrics/XGBOOST/experimento_full/testing_xgboost_MEJORESPARAM10_experimento_full_resample_'+str(resample_item)+'_horizon_'+str(horizon_item)+'_previous_'+str(previous_item)+'.csv')

            list_dfs = []


            d = {'TARGET': predicciones[4]['real'].flatten(), 'FORECAST': predicciones[4]['prediccion'].flatten()}
            df_predicciones = pd.DataFrame(data = d)
            df_predicciones['ESTACION'] = 4

            df_predicciones.to_csv('datos/experimento_full/predicciones_estacion4_MEJORESPARAM10_resample_'+str(resample_item)+'_horizon_'+str(horizon_item)+'_previous_'+str(previous_item)+'.csv')

            print('testing_'+ str(resample_item)+'_'+str(horizon_item)+'_'+ str(previous_item)+' done!')

testing_5_0.25_2 done!
testing_5_0.5_2 done!
testing_5_1_2 done!
testing_5_0.25_7 done!
testing_5_0.5_7 done!
testing_5_1_7 done!
testing_5_0.25_15 done!
testing_5_0.5_15 done!
testing_5_1_15 done!
testing_30_0.25_2 done!
testing_30_0.5_2 done!
testing_30_1_2 done!
testing_30_0.25_7 done!
testing_30_0.5_7 done!
testing_30_1_7 done!
testing_30_0.25_15 done!
testing_30_0.5_15 done!
testing_30_1_15 done!
testing_60_0.25_2 done!
testing_60_0.5_2 done!
testing_60_1_2 done!
testing_60_0.25_7 done!
testing_60_0.5_7 done!
testing_60_1_7 done!
testing_60_0.25_15 done!
testing_60_0.5_15 done!
testing_60_1_15 done!


In [10]:
horizon = [0.25, 0.5, 1]
resample = [5, 30, 60]
previous = [2, 7, 15]

# Start resampling

for resample_item in resample:

    df_train = datos.copy()

    df_train.drop(['ANHO', 'DIA', 'MES', 'HORA', 'MINUTO', 'DIA_TRAF_COD', 'TRAFICO_COD', 'TIPO_COD',
                'MEDICION_DIA', 'MP1_ANTERIOR', 'MP2_5_ANTERIOR', 'MP10_ANTERIOR',
                'TEMPERATURA_PRONOSTICO', 'HUMEDAD_PRONOSTICO', 'PRESION_PRONOSTICO'], axis = 1, inplace = True)

    df_train = df_train.set_index('FECHAHORA', drop = True)

    lista_resample = []

    if resample_item != 5: 

        r_i = str(resample_item) + 'T'

        for station in range(1,11):
            df_aux = df_train[df_train['ESTACION'] == station]

            df_aux = df_aux.resample(r_i).mean()
            lista_resample.append(df_aux)

        df_train = pd.concat(lista_resample)

    df_train['ANHO'] = df_train.index.year
    df_train['MES'] = df_train.index.month
    df_train['HORA'] = df_train.index.hour
    df_train['MINUTO'] = df_train.index.minute
    df_train['DIA'] = df_train.index.day
    df_train['FECHAHORA'] = df_train.index
    df_train.reset_index(inplace = True, drop = True)

    df_train.to_csv('datos/experimento_full/train_resample_'+str(resample_item)+'_minutes.csv')

    # OPTUNA

    for previous_item in previous:
        for horizon_item in horizon:
            
            estacion = 4

            variables = ['MP1', 'MP2_5', 'MP10', 'AQI_MP10', 'AQI_MP2_5', 
                        'PRESION', 'DIA_SEM']

            dependent = ['AQI_MP2_5']

            number_of_features = len(variables)

            training_days = previous_item
            forecast_days = horizon_item

            if resample_item == 5:
                samples_per_day = 288
            elif resample_item == 30:
                samples_per_day = 48
            elif resample_item == 60:
                samples_per_day = 24

            step = forecast_days*samples_per_day

            train_months = relativedelta(months = 12)

            input_samples = int(samples_per_day * training_days) # cantidad de muestras en 7 dias
            output_samples = int(samples_per_day * forecast_days) # cantidad de muestras en 1 dia
            train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test

            X_train, y_train, X_test, y_test = utils_xgboost.get_everything(df_train, 
                                                                estacion,
                                                                train_months, 
                                                                variables, 
                                                                dependent, 
                                                                train_test_samples, 
                                                                input_samples, 
                                                                output_samples, 
                                                                number_of_features,
                                                                step)


            def objective(trial):
    
                params = {
                    'max_depth': trial.suggest_int('max_depth', 1, 9),
                    'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
                    'n_estimators': trial.suggest_int('n_estimators', 10, 300),
                    'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                    'gamma': trial.suggest_loguniform('gamma', 1e-5, 1.0),
                    'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
                    'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
                }
                
                optuna_xgb_model = xgb.XGBRegressor(** params)
                
                trained_xgb_model = MultiOutputRegressor(optuna_xgb_model).fit(X_train , y_train)

                prediction = trained_xgb_model.predict(X_test)

                MAPE = mean_absolute_percentage_error(prediction, y_test)
                print('MAPE: ', MAPE)
                MAE = np.mean(np.abs(prediction - y_test))
                print('MAE: ', MAE)

                MSE = mean_squared_error(prediction, y_test)
                
                return MSE

            study = optuna.create_study(direction='minimize')

            study.optimize(objective, n_trials = 100 )

            trial = study.best_trial

            joblib.dump(study, "optuna_studies/XGBOOST/experimento_full/study_XGBOOST_mejoresparam20_resample_"+str(resample_item)+"_horizon_"+str(horizon_item)+"_previous_"+str(previous_item)+".pkl")

            # VALIDATION

            params = study.best_params

            # study = joblib.load("optuna_studies/XGBOOST/experimento_full/study_XGBOOST_resample_"+str(resample_item)+"_horizon_"+str(horizon_item)+"_previous_"+str(previous_item)+".pkl")

            xgb_model = xgb.XGBRegressor(** params)

            trained_xgb_model = MultiOutputRegressor(xgb_model).fit(X_train , y_train)
            
            prediction = trained_xgb_model.predict(X_test)
            
            predicciones = {}
            metricas = {}
            # guardamos los valores predecidos vs reales en un diccionario
            
            predicciones[4] = {'real' : y_test, 'prediccion': prediction}


            mean_real = y_test.mean()
            mean_prediction = prediction.mean()

            MAPE = mean_absolute_percentage_error(prediction, y_test)
            MAE =100 * mean_absolute_error(prediction, y_test)
            RMSE = mean_squared_error(prediction, y_test, squared = False)
            
            # guardamos las metricas en un diccionario
            
            metricas[4] = {'MAE': MAE, "MAPE": MAPE, 'RMSE': RMSE, 'Media real' : mean_real, 'Media predecida': mean_prediction}

            df_metricas = pd.DataFrame.from_dict(metricas)

            df_metricas.to_csv('metrics/XGBOOST/experimento_full/testing_xgboost_MEJORESPARAM20_experimento_full_resample_'+str(resample_item)+'_horizon_'+str(horizon_item)+'_previous_'+str(previous_item)+'.csv')

            list_dfs = []


            d = {'TARGET': predicciones[4]['real'].flatten(), 'FORECAST': predicciones[4]['prediccion'].flatten()}
            df_predicciones = pd.DataFrame(data = d)
            df_predicciones['ESTACION'] = 4

            df_predicciones.to_csv('datos/experimento_full/predicciones_estacion4_MEJORESPARAM20_resample_'+str(resample_item)+'_horizon_'+str(horizon_item)+'_previous_'+str(previous_item)+'.csv')


[I 2023-04-01 22:51:43,172] A new study created in memory with name: no-name-d472b728-c125-4ecd-a9c5-29bf2e284faa
[I 2023-04-01 22:51:45,672] Trial 0 finished with value: 583.4935874639252 and parameters: {'max_depth': 6, 'learning_rate': 0.010851570643846148, 'n_estimators': 13, 'min_child_weight': 5, 'gamma': 6.143671539913322e-05, 'subsample': 0.29175197066166747, 'colsample_bytree': 0.05630155669347187}. Best is trial 0 with value: 583.4935874639252.


MAPE:  4.30898375748038
MAE:  19.355581740342945


[I 2023-04-01 22:52:19,569] Trial 1 finished with value: 40.31941897286895 and parameters: {'max_depth': 6, 'learning_rate': 0.21465542840581747, 'n_estimators': 263, 'min_child_weight': 4, 'gamma': 0.4457250238703376, 'subsample': 0.09998602917857305, 'colsample_bytree': 0.13619803396426947}. Best is trial 1 with value: 40.31941897286895.


MAPE:  0.4909685655320788
MAE:  4.890513367217035


[I 2023-04-01 22:52:39,207] Trial 2 finished with value: 187.90247521413292 and parameters: {'max_depth': 4, 'learning_rate': 0.7266687095715746, 'n_estimators': 224, 'min_child_weight': 7, 'gamma': 0.5583398742182447, 'subsample': 0.025856506232352902, 'colsample_bytree': 0.018413428072755353}. Best is trial 1 with value: 40.31941897286895.


...

MAPE:  1.0220130423337461
MAE:  9.311156408451444


[I 2023-04-03 09:37:04,350] Trial 60 finished with value: 127.41179098495367 and parameters: {'max_depth': 3, 'learning_rate': 0.02333359309263996, 'n_estimators': 277, 'min_child_weight': 1, 'gamma': 0.04999843743962242, 'subsample': 0.3021055451858249, 'colsample_bytree': 0.24950755923733905}. Best is trial 50 with value: 109.19496717469562.


MAPE:  0.33471076262154464
MAE:  7.629969908480058


[I 2023-04-03 09:43:13,941] Trial 61 finished with value: 108.5238567441184 and parameters: {'max_depth': 2, 'learning_rate': 0.018779706187389816, 'n_estimators': 256, 'min_child_weight': 3, 'gamma': 0.021599588221176264, 'subsample': 0.6805333838286021, 'colsample_bytree': 0.8087401379351337}. Best is trial 61 with value: 108.5238567441184.


MAPE:  0.2932674502573367
MAE:  6.832066567189861
